In [ ]:
import numpy as np
import time

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt
#from tensorflow.examples.tutorials.mnist import input_data

class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1
        
        (self.x_train, self.y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
        self.x_train = self.x_train.reshape(-1, self.img_rows,self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

if __name__ == '__main__':
    mnist_dcgan = MNIST_DCGAN()
    timer = ElapsedTimer()
    mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=500)
    timer.elapsed_time()
    mnist_dcgan.plot_images(fake=True)
    mnist_dcgan.plot_images(fake=False, save2file=True)

11501568/11490434 [==============================] - 1s 0us/step


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-10-08 10:49:08.885412: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-08 10:49:08.885564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)       

2021-10-08 10:49:19.154065: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-08 10:49:19.154887: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-10-08 10:49:19.214875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-08 10:49:20.114310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-08 10:49:22.180919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


0: [D loss: 1.346467, acc: 0.548828]  [A loss: 0.255689, acc: 1.000000]
1: [D loss: 2.066706, acc: 0.500000]  [A loss: 0.126447, acc: 1.000000]
2: [D loss: 1.471735, acc: 0.500000]  [A loss: 0.088165, acc: 1.000000]
3: [D loss: 1.246736, acc: 0.500000]  [A loss: 0.073019, acc: 1.000000]
4: [D loss: 1.054835, acc: 0.500000]  [A loss: 0.057910, acc: 1.000000]
5: [D loss: 0.878411, acc: 0.500000]  [A loss: 0.042886, acc: 1.000000]
6: [D loss: 0.739413, acc: 0.500000]  [A loss: 0.031454, acc: 1.000000]
7: [D loss: 0.625100, acc: 0.500000]  [A loss: 0.023168, acc: 1.000000]
8: [D loss: 0.542052, acc: 0.500000]  [A loss: 0.016721, acc: 1.000000]
9: [D loss: 0.483063, acc: 0.500000]  [A loss: 0.011993, acc: 1.000000]
10: [D loss: 0.435939, acc: 0.500000]  [A loss: 0.009479, acc: 1.000000]
11: [D loss: 0.402830, acc: 0.500000]  [A loss: 0.006464, acc: 1.000000]
12: [D loss: 0.377265, acc: 0.500000]  [A loss: 0.005024, acc: 1.000000]
13: [D loss: 0.359967, acc: 0.500000]  [A loss: 0.003625, acc

...

4838: [D loss: 0.000005, acc: 1.000000]  [A loss: 41.161102, acc: 0.003906]
4839: [D loss: 0.962566, acc: 0.894531]  [A loss: 160.620239, acc: 0.000000]
4840: [D loss: 0.000000, acc: 1.000000]  [A loss: 125.729187, acc: 0.000000]
4841: [D loss: 0.000000, acc: 1.000000]  [A loss: 91.629311, acc: 0.000000]
4842: [D loss: 0.000000, acc: 1.000000]  [A loss: 60.173332, acc: 0.000000]
4843: [D loss: 0.038490, acc: 0.992188]  [A loss: 46.640755, acc: 0.003906]
4844: [D loss: 0.624975, acc: 0.914062]  [A loss: 124.968979, acc: 0.000000]
4845: [D loss: 0.000000, acc: 1.000000]  [A loss: 91.682060, acc: 0.000000]
4846: [D loss: 0.000000, acc: 1.000000]  [A loss: 66.395721, acc: 0.000000]
4847: [D loss: 0.002713, acc: 0.998047]  [A loss: 39.434334, acc: 0.007812]
4848: [D loss: 0.977938, acc: 0.904297]  [A loss: 128.256668, acc: 0.000000]
4849: [D loss: 0.000000, acc: 1.000000]  [A loss: 98.501320, acc: 0.000000]
4850: [D loss: 0.000000, acc: 1.000000]  [A loss: 69.993271, acc: 0.000000]
4851: [D

4946: [D loss: 0.000000, acc: 1.000000]  [A loss: 73.301994, acc: 0.000000]
4947: [D loss: 0.000000, acc: 1.000000]  [A loss: 46.578484, acc: 0.000000]
4948: [D loss: 0.193181, acc: 0.970703]  [A loss: 76.423477, acc: 0.000000]
4949: [D loss: 0.000000, acc: 1.000000]  [A loss: 51.455238, acc: 0.000000]
4950: [D loss: 0.109278, acc: 0.978516]  [A loss: 64.632080, acc: 0.000000]
4951: [D loss: 0.009397, acc: 0.996094]  [A loss: 44.520096, acc: 0.000000]
4952: [D loss: 0.205287, acc: 0.957031]  [A loss: 96.021774, acc: 0.000000]
4953: [D loss: 0.000000, acc: 1.000000]  [A loss: 69.069260, acc: 0.000000]
4954: [D loss: 0.008396, acc: 0.998047]  [A loss: 45.173916, acc: 0.000000]
4955: [D loss: 0.070948, acc: 0.980469]  [A loss: 59.578674, acc: 0.000000]
4956: [D loss: 0.002784, acc: 0.998047]  [A loss: 37.911312, acc: 0.000000]
4957: [D loss: 0.356685, acc: 0.939453]  [A loss: 125.240288, acc: 0.000000]
4958: [D loss: 0.000000, acc: 1.000000]  [A loss: 92.660385, acc: 0.000000]
4959: [D lo

5054: [D loss: 0.172050, acc: 0.947266]  [A loss: 23.712221, acc: 0.000000]
5055: [D loss: 0.117131, acc: 0.974609]  [A loss: 18.308058, acc: 0.000000]
5056: [D loss: 0.247296, acc: 0.939453]  [A loss: 23.801407, acc: 0.000000]
5057: [D loss: 0.112218, acc: 0.966797]  [A loss: 20.449158, acc: 0.000000]
5058: [D loss: 0.186240, acc: 0.947266]  [A loss: 28.425467, acc: 0.000000]
5059: [D loss: 0.022810, acc: 0.990234]  [A loss: 15.484159, acc: 0.003906]
5060: [D loss: 0.322743, acc: 0.902344]  [A loss: 38.383636, acc: 0.000000]
5061: [D loss: 0.000097, acc: 1.000000]  [A loss: 19.674398, acc: 0.000000]
5062: [D loss: 0.235292, acc: 0.923828]  [A loss: 36.157501, acc: 0.000000]
5063: [D loss: 0.047103, acc: 0.988281]  [A loss: 21.880066, acc: 0.000000]
5064: [D loss: 0.502322, acc: 0.888672]  [A loss: 49.085274, acc: 0.000000]
5065: [D loss: 0.000494, acc: 1.000000]  [A loss: 23.746710, acc: 0.000000]
5066: [D loss: 1.617632, acc: 0.728516]  [A loss: 91.283623, acc: 0.000000]
5067: [D los

5162: [D loss: 1.509135, acc: 0.667969]  [A loss: 59.268341, acc: 0.000000]
5163: [D loss: 0.854114, acc: 0.998047]  [A loss: 29.254181, acc: 0.000000]
5164: [D loss: 0.004591, acc: 0.998047]  [A loss: 18.983614, acc: 0.007812]
5165: [D loss: 0.946726, acc: 0.820312]  [A loss: 27.341022, acc: 0.000000]
5166: [D loss: 0.018108, acc: 0.994141]  [A loss: 12.779179, acc: 0.007812]
5167: [D loss: 0.797370, acc: 0.775391]  [A loss: 29.249180, acc: 0.000000]
5168: [D loss: 0.000418, acc: 1.000000]  [A loss: 15.543563, acc: 0.000000]
5169: [D loss: 0.184979, acc: 0.929688]  [A loss: 14.511349, acc: 0.000000]
5170: [D loss: 0.156165, acc: 0.941406]  [A loss: 13.241739, acc: 0.003906]
5171: [D loss: 0.243872, acc: 0.921875]  [A loss: 14.061492, acc: 0.000000]
5172: [D loss: 0.142237, acc: 0.947266]  [A loss: 13.536041, acc: 0.003906]
5173: [D loss: 0.178208, acc: 0.931641]  [A loss: 14.150259, acc: 0.000000]
5174: [D loss: 0.090615, acc: 0.964844]  [A loss: 12.026741, acc: 0.003906]
5175: [D los

5270: [D loss: 0.453865, acc: 0.878906]  [A loss: 35.005543, acc: 0.000000]
5271: [D loss: 0.001530, acc: 1.000000]  [A loss: 16.366693, acc: 0.000000]
5272: [D loss: 0.196668, acc: 0.931641]  [A loss: 22.499134, acc: 0.000000]
5273: [D loss: 0.051688, acc: 0.982422]  [A loss: 15.019690, acc: 0.003906]
5274: [D loss: 0.343221, acc: 0.902344]  [A loss: 31.233236, acc: 0.000000]
5275: [D loss: 0.001626, acc: 1.000000]  [A loss: 14.307892, acc: 0.000000]
5276: [D loss: 0.200204, acc: 0.939453]  [A loss: 21.611334, acc: 0.000000]
5277: [D loss: 0.061890, acc: 0.980469]  [A loss: 16.057533, acc: 0.003906]
5278: [D loss: 0.339594, acc: 0.910156]  [A loss: 31.315231, acc: 0.000000]
5279: [D loss: 0.000506, acc: 1.000000]  [A loss: 15.486403, acc: 0.000000]
5280: [D loss: 0.203061, acc: 0.939453]  [A loss: 23.028214, acc: 0.000000]
5281: [D loss: 0.020505, acc: 0.994141]  [A loss: 13.687804, acc: 0.000000]
5282: [D loss: 0.266394, acc: 0.910156]  [A loss: 30.645237, acc: 0.000000]
5283: [D los

In [7]:
!pip install -q git+https://github.com/tensorflow/examples.git

  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
